In [1]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import logit

%load_ext lab_black
%load_ext watermark

# Priors as hidden mixtures

This example demonstrates ...

Adapted from [unit 9: cumulative2.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/cumulative2.odc).

Associated lecture video: Unit 9 Lesson 8

Use of CUMULATIVE as a check for outliers

How consistent are the data with the model? Check cumulative(y[i], y[i]).

The first y[i] specifies the distribution of y[i]'s, the second is the argument/observation. The output is empirical cdf of y[i]s evaluated at y[i].

Values close to 0 or 1 are potential outliers.

What exactly does the cumulative function in BUGS do????

In [3]:
# fmt: off
y = np.array(
    [0.0, 1.0, 2.0, -1.0, 0.4, -0.5, 0.7, -1.2, 0.1, -0.4, 
     0.2, -0.5, -1.4, 1.8, 0.2, 0.3, -0.6, 1.1, 5.1, -6.3]
)
# fmt: on

In [4]:
with pm.Model() as m:
    theta = pm.Flat("theta")
    log_sigma = pm.Flat("log_sigma")
    tau = pm.Deterministic("tau", 1/exp(2*log_sigma))
    
    cuy = pm.Deterministic("cuy", ???cumulative)
    add = pm.Deterministic("add", logit(cuy)**2)
    a = pm.Deterministic("a", pm.math.sum(add))
    
    pm.Normal("y", mu=theta, tau=tau, observed=y)
    
    trace = pm.sample(2000)

SyntaxError: invalid syntax (917032337.py, line 6)

ERROR:root:Cannot parse: 6:33:     cuy = pm.Deterministic("cuy", ???cumulative)
Traceback (most recent call last):
  File "/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/black/__init__.py", line 1154, in format_str
    dst_contents = _format_str_once(src_contents, mode=mode)
  File "/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/black/__init__.py", line 1164, in _format_str_once
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/black/parsing.py", line 128, in lib2to3_parse
    raise exc from None
black.parsing.InvalidInput: Can

In [ ]:
az.summary(trace)

In [ ]:
%watermark --iversions -v